<a href="https://colab.research.google.com/github/MOPineyro/tp1_18co2024_busqueda_torre_de_hanoi/blob/main/TP1_18Co2024_Busqueda_Torre_de_Hanoi_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Integrantes del Grupo

* Ezequiel Maudet
* Natalia Diaz
* Manuel Pineyro
* Cristian Salinas


En clase presentamos el problema de la torre de Hanoi. Además, vimos diferentes algoritmos de búsqueda que nos permitieron resolver este problema. Para este trabajo práctico, deberán implementar un método de búsqueda para resolver con 5 discos, del estado inicial y objetivo.


Tareas y preguntas a resolver:

1. ¿Cuáles son los PEAS de este problema? (Performance, Environment, Actuators, Sensors).

2. ¿Cuáles son las propiedades del entorno de trabajo?

3. En el contexto de este problema, establezca cuáles son los: estado, espacio de estados, árbol de búsqueda, nodo de búsqueda, objetivo, acción y frontera.
4. Implemente algún método de búsqueda. Puedes elegir cualquiera menos búsqueda en anchura primero (el desarrollado en clase). Sos libre de elegir cualquiera de los vistos en clases, o inclusive buscar nuevos.
5. A nivel implementación, ¿qué tiempo y memoria ocupa el algoritmo? (Se recomienda correr 10 veces y calcular promedio y desvío estándar de las métricas).
6. Si la solución óptima es 2k -1 movimientos con k igual al número de discos. Qué tan lejos está la solución del algoritmo implementado de esta solución óptima (se recomienda correr al menos 10 veces y usar el promedio de trayecto usado).

***El entregable es:***
* Un archivo de txt/PDF/Word con las respuestas
https://docs.google.com/document/d/1AXsLMskZXxzqyHd3vTW7I5g6kdWO2sSuVSk63FiKFOM/edit?usp=sharing

* Los archivos con el código implementado, tambien pueden enviar una Notebook con el contenido y la solución.


 Si además agregan los json para usar en el simulador, es mejor. Pueden subir el contenido o proporcionar un enlace a un repositorio público (GitHub o GitLab) con el contenido. No olvidar especificar en el entregable los autores del TP.

Para resolver este TP son libres de usar los recursos que crean necesarios. Pueden resolverlo en cualquier lenguaje de programación y de la forma que consideren apropiada.

Pueden ahorrar tiempo usando el código ya implementado en Python que se encuentra en el repositorio hanoi_tower. Si usan este código, solo deben implementar el algoritmo de búsqueda, pero es importante que lean el código y entiendan que es cada parte

# Backend de Hanoi

## Dependencia aima.py

In [2]:
"""
Código de Python del libro Artificial Intelligence: A Modern Approach. Implementa clases y funciones primitivas que
permite usar en nuestros programas.

https://github.com/aimacode
https://github.com/aimacode/aima-python

The MIT License (MIT)

Copyright (c) 2016 aima-python contributors

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated
documentation files (the "Software"), to deal in the Software without restriction, including without limitation the
rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit
persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the
Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE
WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR
COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR
OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.
"""
import heapq


def is_in(elt, seq):
    """Similar to (elt in seq), but compares with 'is', not '=='."""
    return any(x is elt for x in seq)


class Problem:
    """The abstract class for a formal problem. You should subclass
    this and implement the methods actions and result, and possibly
    __init__, goal_test, and path_cost. Then you will create instances
    of your subclass and solve them with the various search functions."""

    def __init__(self, initial, goal=None):
        """The constructor specifies the initial state, and possibly a goal
        state, if there is a unique goal. Your subclass's constructor can add
        other arguments."""
        self.initial = initial
        self.goal = goal

    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""
        raise NotImplementedError

    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        raise NotImplementedError

    def goal_test(self, state):
        """Return True if the state is a goal. The default method compares the
        state to self.goal or checks for state in self.goal if it is a
        list, as specified in the constructor. Override this method if
        checking against a single self.goal is not enough."""
        if isinstance(self.goal, list):
            return is_in(state, self.goal)
        else:
            return state == self.goal

    def path_cost(self, c, state1, action, state2):
        """Return the cost of a solution path that arrives at state2 from
        state1 via action, assuming cost c to get up to state1. If the problem
        is such that the path doesn't matter, this function will only look at
        state2. If the path does matter, it will consider c and maybe state1
        and action. The default method costs 1 for every step in the path."""
        return c + 1

    def value(self, state):
        """For optimization problems, each state has a value. Hill Climbing
        and related algorithms try to maximize this value."""
        raise NotImplementedError


# ______________________________________________________________________________


class Node:
    """A node in a search tree. Contains a pointer to the parent (the node
    that this is a successor of) and to the actual state for this node. Note
    that if a state is arrived at by two paths, then there are two nodes with
    the same state. Also includes the action that got us to this state, and
    the total path_cost (also known as g) to reach the node. Other functions
    may add an f and h value; see best_first_graph_search and astar_search for
    an explanation of how the f and h values are handled. You will not need to
    subclass this class."""

    def __init__(self, state, parent=None, action=None, path_cost=0):
        """Create a search tree Node, derived from a parent by an action."""
        self.state = state
        self.parent = parent
        self.action = action
        self.path_cost = path_cost
        self.depth = 0
        if parent:
            self.depth = parent.depth + 1

    def __repr__(self):
        return "<Node {}>".format(self.state)

    def __lt__(self, node):
        return self.state < node.state

    def expand(self, problem):
        """List the nodes reachable in one step from this node."""
        return [self.child_node(problem, action)
                for action in problem.actions(self.state)]

    def child_node(self, problem, action):
        """[Figure 3.10]"""
        next_state = problem.result(self.state, action)
        next_node = Node(next_state, self, action, problem.path_cost(self.path_cost, self.state, action, next_state))
        return next_node

    def solution(self):
        """Return the sequence of actions to go from the root to this node."""
        return [node.action for node in self.path()[1:]]

    def path(self):
        """Return a list of nodes forming the path from the root to this node."""
        node, path_back = self, []
        while node:
            path_back.append(node)
            node = node.parent
        return list(reversed(path_back))

    # We want for a queue of nodes in breadth_first_graph_search or
    # astar_search to have no duplicated states, so we treat nodes
    # with the same state as equal. [Problem: this may not be what you
    # want in other contexts.]

    def __eq__(self, other):
        return isinstance(other, Node) and self.state == other.state

    def __hash__(self):
        # We use the hash value of the state
        # stored in the node instead of the node
        # object itself to quickly search a node
        # with the same state in a Hash Table
        return hash(self.state)


# ______________________________________________________________________________

class PriorityQueue:
    """A Queue in which the minimum (or maximum) element (as determined by f and
    order) is returned first.
    If order is 'min', the item with minimum f(x) is
    returned first; if order is 'max', then it is the item with maximum f(x).
    Also supports dict-like lookup."""

    def __init__(self, order='min', f=lambda x: x):
        self.heap = []
        if order == 'min':
            self.f = f
        elif order == 'max':  # now item with max f(x)
            self.f = lambda x: -f(x)  # will be popped first
        else:
            raise ValueError("Order must be either 'min' or 'max'.")

    def append(self, item):
        """Insert item at its correct position."""
        heapq.heappush(self.heap, (self.f(item), item))

    def extend(self, items):
        """Insert each item in items at its correct position."""
        for item in items:
            self.append(item)

    def pop(self):
        """Pop and return the item (with min or max f(x) value)
        depending on the order."""
        if self.heap:
            return heapq.heappop(self.heap)[1]
        else:
            raise Exception('Trying to pop from empty PriorityQueue.')

    def __len__(self):
        """Return current capacity of PriorityQueue."""
        return len(self.heap)

    def __contains__(self, key):
        """Return True if the key is in PriorityQueue."""
        return any([item == key for _, item in self.heap])

    def __getitem__(self, key):
        """Returns the first value associated with key in PriorityQueue.
        Raises KeyError if key is not present."""
        for value, item in self.heap:
            if item == key:
                return value
        raise KeyError(str(key) + " is not in the priority queue")

    def __delitem__(self, key):
        """Delete the first occurrence of key."""
        try:
            del self.heap[[item == key for _, item in self.heap].index(True)]
        except ValueError:
            raise KeyError(str(key) + " is not in the priority queue")
        heapq.heapify(self.heap)



## Dependencia hanoi_state.py

In [3]:
import copy
from typing import Optional

#import aima


def is_sorted(test_list: list) -> bool:
    """
    Comprueba si una lista está ordenada de forma descendente.

    Args:
        test_list (list): Lista a comprobar.

    Returns:
        bool: True si la lista está ordenada de forma descendente, False en caso contrario.
    """
    if test_list == sorted(test_list, reverse=True):
        return True
    return False


class StatesHanoi:
    """
    Representa un estado posible de ubicación de discos de la Torre de Hanoi.
    """

    def __init__(self, rod1: list, rod2: list, rod3: list, max_disks: int = 5, cost: float = 0.0):
        """
        Inicializa un estado posible de ubicación de discos de la Torre de Hanoi.

        Args:
            rod1 (list): Discos en la primera varilla.
            rod2 (list): Discos en la segunda varilla.
            rod3 (list): Discos en la tercera varilla.
            max_disks (int): Máximo número de discos permitidos.
            cost (float): Costo asociado al estado.
        """
        # Comprobamos si es un estado ilegal
        if (set.intersection(set(rod1), set(rod2)) or
                set.intersection(set(rod2), set(rod3)) or
                set.intersection(set(rod1), set(rod3))):
            raise ValueError('El mismo disco está en varillas diferentes')

        all_values = set.union(set(rod1), set(rod2), set(rod3))
        if not all(0 < i < (max_disks + 1) for i in all_values):
            raise ValueError('Valor de disco incorrecto')

        if not all(i in all_values for i in range(1, max_disks + 1)):
            raise ValueError('No todos los discos están insertados')

        for rod in [rod1, rod2, rod3]:
            if not is_sorted(rod):
                raise ValueError('No es un estado de Hanoi válido')

        self.rods = [rod1, rod2, rod3]
        self.number_of_disks = sum([len(rod) for rod in self.rods])
        self.number_of_pegs = 3
        self.accumulated_cost = cost

        self.string_representation = ""
        self.generate_representation()

    def generate_representation(self):
        """
        Genera una representación en forma de string del estado de Hanoi.
        """
        strings = 'HanoiState: '
        for rod in self.rods:
            strings += ' '.join(str(disk) for disk in rod)
            strings += " | "
        self.string_representation = strings[:-3]

    def __eq__(self, other):
        """
        Compara dos estados de Hanoi para verificar si son iguales.

        Dos estados de Hanoi son iguales si tienen la misma cantidad de discos y la misma ubicación.

        Args:
            other: Otro estado de Hanoi a comparar.

        Returns:
            bool: True si los estados son iguales, False en caso contrario.
        """
        if self.number_of_disks == other.number_of_disks:
            if self.rods == other.rods:
                return True

        return False

    def __lt__(self, other):
        """
        Compara dos estados de Hanoi para verificar si uno es mayor que el otro.

        Esto se determina con el costo acumulado, quien tiene un costo mayor es mas grande

        Args:
            other: Otro estado de Hanoi a comparar.

        Returns:
            bool: True si los estados son iguales, False en caso contrario.
        """
        return self.accumulated_cost < other.accumulated_cost

    def __repr__(self):
        """
        Representación formal de un objeto StatesHanoi.

        Returns:
            str: Cadena que representa el estado de Hanoi.
        """
        self.generate_representation()
        return self.string_representation

    def __str__(self):
        """
        Representación en string de un objeto StatesHanoi.

        Returns:
            str: Cadena que representa el estado de Hanoi.
        """
        self.generate_representation()
        return self.string_representation

    def __hash__(self):
        """
        Genera un hash para el objeto StatesHanoi.

        Returns:
            int: Hash generado para el estado de Hanoi.
        """
        self.generate_representation()
        return hash(self.string_representation)

    def get_last_disk_rod(self, number_rod: int, peek: bool = False) -> Optional[int]:
        """
        Obtiene el último disco de una varilla específica.

        Args:
            number_rod (int): Índice de la varilla.
            peek (bool): Indica si se desea solo obtener el último disco sin eliminarlo de la varilla.

        Returns:
            Optional[int]: El último disco de la varilla si existe, None en caso contrario.
        """
        rod = self.rods[number_rod]
        if len(rod) != 0:
            if peek:
                return rod[-1]
            return rod.pop()
        return None

    def check_valid_disk_in_rod(self, number_rod: int, disk: int) -> bool:
        """
        Comprueba si es válido colocar un disco en una varilla específica.

        Args:
            number_rod (int): Índice de la varilla.
            disk (int): Número del disco a colocar.

        Returns:
            bool: True si es válido colocar el disco en la varilla, False en caso contrario.
        """
        last_disk_in_rod = self.get_last_disk_rod(number_rod, peek=True)
        if last_disk_in_rod:
            if last_disk_in_rod > disk:
                return True
        else:
            return True
        return False

    def put_disk_in_rod(self, number_rod: int, disk: int):
        """
        Coloca un disco en una varilla específica.

        Args:
            number_rod (int): Índice de la varilla.
            disk (int): Número del disco a colocar.
        """
        if self.check_valid_disk_in_rod(number_rod, disk):
            self.rods[number_rod].append(disk)

    def accumulate_cost(self, cost):
        """
        Acumula el costo asociado al estado.

        Args:
            cost: Costo a acumular.
        """
        self.accumulated_cost += cost

    def get_accumulated_cost(self):
        """
        Obtiene el costo acumulado del estado.

        Returns:
            float: Costo acumulado del estado.
        """
        return self.accumulated_cost

    def get_state(self) -> list:
        """
        Obtiene una representación del estado de Hanoi.

        Returns:
            list: Lista que representa el estado de Hanoi.
        """
        return self.rods

    def get_state_dict(self) -> dict:
        """
        Obtiene una representación del estado de Hanoi como un diccionario.

        Returns:
            dict: Diccionario que representa el estado de Hanoi.
        """
        return_dict = {}
        for index, rod in enumerate(self.rods):
            return_dict[f'peg_{index+1}'] = rod
        return return_dict


class ActionHanoi:
    """
    Representa una acción en el problema de la Torre de Hanoi.
    """

    def __init__(self, disk: int, rod_input: int, rod_out: int):
        """
        Inicializa una acción para mover un disco de la Torre de Hanoi.

        Args:
            disk (int): Número del disco.
            rod_input (int): Índice de la varilla de entrada.
            rod_out (int): Índice de la varilla de salida.
        """
        self.disk = disk
        self.rod_input = rod_input

        if rod_input != rod_out:
            self.action = f"Move disk {disk} from {rod_input + 1} to {rod_out + 1}"
            self.action_dict = {
                "type": "movement",
                "disk": disk,
                "peg_start": rod_input + 1,
                "peg_end": rod_out + 1
            }
            self.cost = 1.0
            self.rod_out = rod_out
        else:
            self.action = f"Maintain disk {disk} in {rod_input + 1}"
            self.action_dict = {
                "type": "maintain",
                "disk": disk,
                "peg": rod_input + 1
            }
            self.cost = 0.0
            self.rod_out = rod_input

    def __repr__(self):
        """
        Representación formal de una acción.

        Returns:
            str: Cadena que representa la acción.
        """
        return self.action

    def __str__(self):
        """
        Representación en cadena de una acción.

        Returns:
            str: String que representa la acción.
        """
        return self.action

    def execute(self, state_hanoi: StatesHanoi):
        """
        Ejecuta la acción en un estado de Hanoi dado.

        Args:
            state_hanoi (StatesHanoi): Estado de Hanoi en el que se ejecutará la acción.

        Returns:
            StatesHanoi: Nuevo estado de Hanoi después de ejecutar la acción.
        """
        if "move" in self.action.lower():
            state_out = copy.deepcopy(state_hanoi)

            disk = state_out.get_last_disk_rod(self.rod_input)
            state_out.put_disk_in_rod(self.rod_out, disk)
            state_out.accumulate_cost(self.cost)
            return state_out
        return state_hanoi


class ProblemHanoi(Problem):
    """
    Clase que define el problema de la Torre de Hanoi.

    Attributes:
        initial (hanoi_states.StatesHanoi): El estado inicial del problema.
        goal (hanoi_states.StatesHanoi): El estado objetivo del problema.
    """

    def __init__(self, initial: StatesHanoi, goal: StatesHanoi):
        """
        Inicializa el problema de la Torre de Hanoi.

        Args:
            initial (StatesHanoi): El estado inicial del problema.
            goal (StatesHanoi): El estado objetivo del problema.
        """
        super().__init__(initial=initial, goal=goal)

    def actions(self, state: StatesHanoi):
        """
        Devuelve todas las acciones posibles que se pueden ejecutar desde un estado dado.

        Args:
            state (StatesHanoi): Estado actual de la Torre de Hanoi.

        Returns:
            list: Lista con todas las acciones posibles.
        """
        actions_list = []
        for i in range(3):
            for j in range(3):
                disk = state.get_last_disk_rod(i, peek=True)
                if disk:
                    if state.check_valid_disk_in_rod(j, disk):
                        actions_list.append(ActionHanoi(disk, i, j))
                else:
                    break

        return actions_list

    def result(self, state: StatesHanoi, action: ActionHanoi):
        """
        Calcula el nuevo estado después de aplicar una acción.

        Args:
            state (hanoi_states.StatesHanoi): Estado actual de la Torre de Hanoi.
            action (hanoi_states.ActionHanoi): Acción a aplicar.

        Returns:
            hanoi_states.StatesHanoi: Nuevo estado después de aplicar la acción.
        """
        return action.execute(state)

    def path_cost(self, c, state1, action, state2):
        """
        Calcula el costo del camino.

        Args:
            c: Costo acumulado hasta el estado actual (No utilizado, pero necesario por la herencia)
            state1 (hanoi_states.StatesHanoi): Estado inicial.
            action (hanoi_states.ActionHanoi): Acción realizada.
            state2 (hanoi_states.StatesHanoi): Estado resultante después de la acción. (No utilizado, pero necesario
            por la herencia)

        Returns:
            float: Costo total del camino.
        """
        return state1.accumulated_cost + action.cost


## Dependencia NodeHanoi

In [4]:
import json



class NodeHanoi(Node):
    """
    Clase que define un nodo en el arbol de búsqueda para la Torre de Hanoi.
    """

    def __init__(self, state: StatesHanoi, parent=None, action=None):
        """
        Inicializa un nodo en el espacio de búsqueda.

        Args:
            state (hanoi_states.StatesHanoi): Estado del nodo.
            parent (hanoi_states.StatesHanoi | None): Nodo padre.
            action (hanoi_states.ActionHanoi | None): Acción realizada para llegar a este nodo.
        """
        super().__init__(state, parent=parent, action=action)
        self.path_cost = state.accumulated_cost

    def child_node(self, problem: ProblemHanoi, action: ActionHanoi):
        """
        Genera el nodo hijo a partir de una acción.

        Args:
            problem (hanoi_states.ProblemHanoi): Problema de la Torre de Hanoi.
            action (hanoi_states.ActionHanoi): Acción a aplicar.

        Returns:
            NodeHanoi: Nodo hijo generado.
        """
        next_state = problem.result(self.state, action)
        next_node = NodeHanoi(next_state, parent=self, action=action)
        return next_node

    def generate_solution_for_simulator(self, initial_state_file="./initial_state.json",
                                        sequence_file="./sequence.json"):
        """
        Genera los archivos JSON para el simulador de la Torre de Hanoi.

        Args:
            initial_state_file (str): Ruta del archivo JSON para el estado inicial.
            sequence_file (str): Ruta del archivo JSON para la secuencia de movimientos.
        """
        list_solution = self.path()

        with open(initial_state_file, "w") as file:
            initial_state = list_solution[0].state.get_state_dict()
            json.dump(initial_state, file)

        with open(sequence_file, "w") as file:
            sequence = [node.action.action_dict for node in list_solution[1:]]
            json.dump(sequence, file, indent=2)


# Busqueda en Profundidad - LIFO



In [5]:
import pandas as pd
import timeit

In [6]:
# Inicializamos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una pila LIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la pila no este vacía
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el último nodo de la pila (LIFO)

    # Agregamos el nodo al set para evitar duplicados
    explored.add(node.state)

    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        print("Encontramos la solución")
        break

    # Agregamos a la pila todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.append(next_node)  # Usamos append para agregar los sucesores al final (LIFO)****************************


Encontramos la solución


In [8]:
print(f'Longitud del camino de la solución: {last_node.state.accumulated_cost}')

Longitud del camino de la solución: 31.0


In [9]:
solucion_prof = {'Long_Solucion': last_node.state.accumulated_cost,
                 'Explorados':len(explored),
                 'Fronteras': len(frontier)}
solucion_prof

{'Long_Solucion': 31.0, 'Explorados': 233, 'Fronteras': 285}

In [10]:
print(len(explored), "nodos se expandieron y", len(frontier), "nodos quedaron en la frontera")

233 nodos se expandieron y 285 nodos quedaron en la frontera


In [11]:
node = last_node
while node.parent is not None:
    print(node.state)
    node = node.parent

HanoiState:  |  | 5 4 3 2 1
HanoiState: 1 |  | 5 4 3 2
HanoiState: 1 | 2 | 5 4 3
HanoiState:  | 2 1 | 5 4 3
HanoiState: 3 | 2 1 | 5 4
HanoiState: 3 | 2 | 5 4 1
HanoiState: 3 2 |  | 5 4 1
HanoiState: 3 2 1 |  | 5 4
HanoiState: 3 2 1 | 4 | 5
HanoiState: 3 2 | 4 1 | 5
HanoiState: 3 | 4 1 | 5 2
HanoiState: 3 | 4 | 5 2 1
HanoiState:  | 4 3 | 5 2 1
HanoiState: 1 | 4 3 | 5 2
HanoiState: 1 | 4 3 2 | 5
HanoiState:  | 4 3 2 1 | 5
HanoiState: 5 | 4 3 2 1 | 
HanoiState: 5 | 4 3 2 | 1
HanoiState: 5 2 | 4 3 | 1
HanoiState: 5 2 1 | 4 3 | 
HanoiState: 5 2 1 | 4 | 3
HanoiState: 5 2 | 4 1 | 3
HanoiState: 5 | 4 1 | 3 2
HanoiState: 5 | 4 | 3 2 1
HanoiState: 5 4 |  | 3 2 1
HanoiState: 5 4 1 |  | 3 2
HanoiState: 5 4 1 | 2 | 3
HanoiState: 5 4 | 2 1 | 3
HanoiState: 5 4 3 | 2 1 | 
HanoiState: 5 4 3 | 2 | 1
HanoiState: 5 4 3 2 |  | 1


In [12]:
def hanoi_solver():
  # Inicializamos el problema
  initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
  goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
  problem = ProblemHanoi(initial=initial_state, goal=goal_state)

  frontier = [NodeHanoi(problem.initial)]  # Creamos una pila LIFO con el nodo inicial

  explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

  # Mientras que la pila no este vacía
  while len(frontier) != 0:
      node = frontier.pop()  # Extraemos el último nodo de la pila (LIFO)

      # Agregamos el nodo al set para evitar duplicados
      explored.add(node.state)

      if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
          last_node = node
          break

      # Agregamos a la pila todos los nodos sucesores del nodo actual
      for next_node in node.expand(problem):
          # Solo si no fue explorado
          if next_node.state not in explored:
              frontier.append(next_node)  # Usamos append para agregar los sucesores al final (LIFO)

In [13]:
# Lista para almacenar los tiempos
execution_times = []

# Ejecutamos 10 veces y almacenamos cada tiempo en la lista
for _ in range(10):
    execution_time = timeit.timeit(hanoi_solver, number=1)
    execution_times.append(execution_time)

# Imprimimos los tiempos de ejecución
print("Execution times for 10 runs:", execution_times)

Execution times for 10 runs: [0.021534608999985494, 0.013288461000001917, 0.012104148999981135, 0.014159347999992633, 0.01506470600000398, 0.015894103999983145, 0.011997374000003447, 0.012904759000008426, 0.012079913000007991, 0.011981959999985747]


In [14]:
solucion_prof['tiempo']= pd.Series(execution_times).mean()
solucion_prof['desvio_tiempo']= pd.Series(execution_times).std()
solucion_prof

{'Long_Solucion': 31.0,
 'Explorados': 233,
 'Fronteras': 285,
 'tiempo': 0.014100938299995392,
 'desvio_tiempo': 0.0029530566090975497}

In [15]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

# Inicializamos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una pila LIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la pila no este vacía
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el último nodo de la pila (LIFO)

    # Agregamos el nodo al set para evitar duplicados
    explored.add(node.state)

    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        break

    # Agregamos a la pila todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.append(next_node)  # Usamos append para agregar los sucesores al final (LIFO)

_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

print(f"Maxima memoria ocupada: {round(memory_peak, 2)} [MB]", )

Maxima memoria ocupada: 0.26 [MB]


In [16]:
solucion_prof['memoria']= memory_peak
solucion_prof

{'Long_Solucion': 31.0,
 'Explorados': 233,
 'Fronteras': 285,
 'tiempo': 0.014100938299995392,
 'desvio_tiempo': 0.0029530566090975497,
 'memoria': 0.25505828857421875}

In [17]:
#LIFO
pd.DataFrame(solucion_prof, index=[0])

,Long_Solucion,Explorados,Fronteras,tiempo,desvio_tiempo,memoria
0,31.0,233,285,0.014101,0.002953,0.255058


# Busque en Anchura - FIFO

In [18]:
# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola

    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)

    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        print("Encontramos la solución")
        break

    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)



Encontramos la solución


In [19]:
solucion_Anch = {'Long_Solucion': last_node.state.accumulated_cost,
                 'Explorados':len(explored),
                 'Fronteras': len(frontier)}
solucion_Anch

{'Long_Solucion': 31.0, 'Explorados': 233, 'Fronteras': 285}

In [20]:
def hanoi_solver():
  # Inicializaos el problema
  initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
  goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
  problem = ProblemHanoi(initial=initial_state, goal=goal_state)

  frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

  explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

  # Mientras que la cola no este vacia
  while len(frontier) != 0:
      node = frontier.pop()  # Extraemos el primer nodo de la cola

      # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
      explored.add(node.state)

      if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
          last_node = node
          break

      # Agregamos a la cola todos los nodos sucesores del nodo actual
      for next_node in node.expand(problem):
          # Solo si no fue explorado
          if next_node.state not in explored:
              frontier.insert(0, next_node)


In [21]:
# Lista para almacenar los tiempos
execution_times = []

# Ejecutamos 10 veces y almacenamos cada tiempo en la lista
for _ in range(10):
    execution_time = timeit.timeit(hanoi_solver, number=1)
    execution_times.append(execution_time)

# Imprimimos los tiempos de ejecución
print("Execution times for 10 runs:", execution_times)

Execution times for 10 runs: [0.16121381299998916, 0.17121378500002038, 0.14439660100001106, 0.16794115499999407, 0.16650393599999802, 0.16477701900001307, 0.14848714599997948, 0.15736525700000925, 0.1409590279999975, 0.13949420500000542]


In [22]:
solucion_Anch['tiempo']= pd.Series(execution_times).mean()
solucion_Anch['desvio_tiempo']= pd.Series(execution_times).std()
solucion_Anch

{'Long_Solucion': 31.0,
 'Explorados': 233,
 'Fronteras': 285,
 'tiempo': 0.15623519450000173,
 'desvio_tiempo': 0.011924147907089244}

In [23]:
import tracemalloc

# Para medir memoria consumida (usamos el pico de memoria)
tracemalloc.start()

# Inicializaos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

frontier = [NodeHanoi(problem.initial)]  # Creamos una cola FIFO con el nodo inicial

explored = set()  # Este set nos permite ver si ya exploramos un estado para evitar repetir indefinidamente

# Mientras que la cola no este vacia
while len(frontier) != 0:
    node = frontier.pop()  # Extraemos el primer nodo de la cola

    # Agregamos nodo al set. Esto evita guardar duplicados, porque set nunca tiene elementos repetidos
    explored.add(node.state)

    if problem.goal_test(node.state):  # Comprobamos si hemos alcanzado el estado objetivo
        last_node = node
        break

    # Agregamos a la cola todos los nodos sucesores del nodo actual
    for next_node in node.expand(problem):
        # Solo si no fue explorado
        if next_node.state not in explored:
            frontier.insert(0, next_node)

_, memory_peak = tracemalloc.get_traced_memory()
memory_peak /= 1024*1024
tracemalloc.stop()

In [24]:
solucion_Anch['memoria']= memory_peak
solucion_Anch

{'Long_Solucion': 31.0,
 'Explorados': 233,
 'Fronteras': 285,
 'tiempo': 0.15623519450000173,
 'desvio_tiempo': 0.011924147907089244,
 'memoria': 1.621530532836914}

In [25]:
pd.DataFrame(solucion_Anch, index=[0])

,Long_Solucion,Explorados,Fronteras,tiempo,desvio_tiempo,memoria
0,31.0,233,285,0.156235,0.011924,1.621531


In [26]:
pd.DataFrame({"FIFO":solucion_Anch,
              "LIFO": solucion_prof})

,FIFO,LIFO
Long_Solucion,31.000000,31.000000
Explorados,233.000000,233.000000
Fronteras,285.000000,285.000000
tiempo,0.156235,0.014101
desvio_tiempo,0.011924,0.002953
memoria,1.621531,0.255058


# Búsqueda A*

### Setup

In [27]:
import time
import statistics
import tracemalloc

def measure_algorithm(algorithm, num_executions=10):
    times = []
    memories = []
    solution_lengths = []
    explored_counts = []
    frontier_counts = []

    for _ in range(num_executions):
        initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
        goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
        problem = ProblemHanoi(initial=initial_state, goal=goal_state)

        start_time = time.time()
        tracemalloc.start()

        result = algorithm(problem)

        end_time = time.time()
        _, memory_peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()

        times.append(end_time - start_time)
        memories.append(memory_peak / (1024 * 1024))  # Convert to MB

        if isinstance(result, tuple):
            last_node, explored, frontier = result
        else:
            last_node = result
            explored = getattr(last_node, 'explored', set())
            frontier = getattr(last_node, 'frontier', [])

        solution_lengths.append(last_node.state.accumulated_cost)
        explored_counts.append(len(explored))
        frontier_counts.append(len(frontier))

    return {
        'Long_Solucion': statistics.mean(solution_lengths),
        'Explorados': statistics.mean(explored_counts),
        'Fronteras': statistics.mean(frontier_counts),
        'tiempo': f"{statistics.mean(times)*1000:.2f} ms ± {statistics.stdev(times)*1000:.2f} ms",
        'memoria': f"{statistics.mean(memories):.6f} MB ± {statistics.stdev(memories):.6f} MB"
    }

# Inicializamos el problema
initial_state = StatesHanoi([5, 4, 3, 2, 1], [], [], max_disks=5)
goal_state = StatesHanoi([], [], [5, 4, 3, 2, 1], max_disks=5)
problem = ProblemHanoi(initial=initial_state, goal=goal_state)

### A*

In [28]:
import heapq

def heuristic(state, num_disks):
    # Calcula una estimación de cuántos movimientos faltan para llegar al objetivo
    count = sum(len(peg) for peg_index, peg in enumerate(state.rods) if peg_index != 2)
    return (2 ** count) - 1 if count > 0 else 0

def a_star_search(problem):
    initial_node = NodeHanoi(problem.initial)
    initial_node.state.heuristic = heuristic(initial_node.state, initial_node.state.number_of_disks)
    frontier = [(initial_node.state.accumulated_cost + initial_node.state.heuristic, initial_node)]
    frontier_set = {initial_node.state}
    explored = set()

    while frontier:
        _, node = heapq.heappop(frontier)
        frontier_set.remove(node.state)

        if problem.goal_test(node.state):
            return node, explored, frontier_set

        explored.add(node.state)

        for child in node.expand(problem):
            if child.state not in explored and child.state not in frontier_set:
                child.state.heuristic = heuristic(child.state, child.state.number_of_disks)
                heapq.heappush(frontier, (child.state.accumulated_cost + child.state.heuristic, child))
                frontier_set.add(child.state)
            elif child.state in frontier_set:
                index = next(i for i, (_, n) in enumerate(frontier) if n.state == child.state)
                if child.state.accumulated_cost < frontier[index][1].state.accumulated_cost:
                    frontier[index] = (child.state.accumulated_cost + child.state.heuristic, child)
                    heapq.heapify(frontier)

    return None, explored, frontier_set

# Ejecutamos A*
last_node = a_star_search(problem)

print("Encontramos la solución" if last_node else "No se encontró solución")

Encontramos la solución


In [29]:
solucion_AStar = measure_algorithm(a_star_search)
pd.DataFrame(solucion_AStar, index=[0])


,Long_Solucion,Explorados,Fronteras,tiempo,memoria
0,31.0,161,9,99.19 ms ± 29.08 ms,0.200412 MB ± 0.014591 MB


In [30]:
combined_results = pd.DataFrame({
    'LIFO (Profundidad)': solucion_prof,
    'FIFO (Anchura)': solucion_Anch,
    'A*': solucion_AStar
}).T
combined_results

,Long_Solucion,Explorados,Fronteras,tiempo,desvio_tiempo,memoria
LIFO (Profundidad),31.0,233.0,285.0,0.014101,0.002953,0.255058
FIFO (Anchura),31.0,233.0,285.0,0.156235,0.011924,1.621531
A*,31.0,161,9,99.19 ms ± 29.08 ms,NaN,0.200412 MB ± 0.014591 MB


**LIFO (búsqueda en profundidad): Mas Rapida, Usa Menos Memoria pero la solucion no es la optima.**

Ventajas: Menor uso de memoria, útil para soluciones profundas, más rápida para encontrar cualquier solución, fácil implementación recursiva.
Desventajas: No garantiza la solución óptima y puede quedarse atrapada en ramas infinitas.

**FIFO (búsqueda en anchura): Mas lenta, Usa Mas Memoria pero la solucion es la optima.**

Ventajas: Garantiza encontrar la solución más corta (óptima) y no se queda atrapada en ramas infinitas.
Desventajas: Usa más memoria y puede ser más lenta si la solución está en un nivel profundo.
La elección entre LIFO y FIFO dependerá del tipo de problema que estés resolviendo y si te interesa encontrar la solución óptima o simplemente cualquier solución rápida.

**A* (A-star): Combina las ventajas de búsqueda en profundidad y anchura, generalmente más eficiente.**

Ventajas: Encuentra la solución óptima si existe. Generalmente más eficiente que FIFO en términos de nodos explorados. Puede ser más rápido que FIFO y LIFO en muchos casos. Utiliza una función heurística para guiar la búsqueda. Desventajas: Requiere una buena función heurística para ser eficiente. Puede usar más memoria que LIFO (pero generalmente menos que FIFO). La implementación puede ser más compleja que LIFO o FIFO.